## <font color='darkblue'>FP Terminology (Imperative vs Declarative)</font>
* [Comparison of programming paradigms](https://en.wikipedia.org/wiki/Comparison_of_programming_paradigms)

### <font color='darkgreen'>Imperative</font>
<b><font color='darkblue' size='3ptx'>Imperative programming</font></b> is like building assembly lines, which take some initial global state as raw material, apply various specific transformations, mutations to it as this material is pushed through the line, and at the end comes off the end product, the final global state, that represents the result of the computation. Each step needs to change, rotate, massage the workpiece precisely one specific way, so that it is prepared for subsequent steps downstream. Every step downstream depend on every previous step, and their order is therefore fixed and rigid. Because of these dependencies, an individual computational step has not much use and meaning in itself, but only in the context of all the others, and to understand it, one must understand how the whole line works.


In [1]:
salaries = [
  # Is male,  salary
  (True,       9000),
  (False,    1_2000),
  (False,      6000),
  (True,     1_4000),
]

def imperative_way(salaries):
  """Calculates the sum of salaries for male and female.
  
  Args:
    salaries: List of tuple(is male, salary)
    
  Returns:
    tuple(sum of male's salary, sum of female's salary)
  """
  female_sum = male_sum = 0
  for is_male, salary in salaries:
    if is_male:
      male_sum += salary
    else:
      female_sum += salary
      
  return (male_sum, female_sum)

In [2]:
imperative_way(salaries)

(23000, 18000)

### <font color='darkgreen'>Declarative</font>
<b><font size='3ptx' color='darkblue'>[declarative programming](https://en.wikipedia.org/wiki/Declarative_programming)</font></b> is a programming paradigm—a style of building the structure and elements of computer programs—that expresses the logic of a computation without describing its control flow.

In [3]:
def declarative_way(salaries):
  """Calculates the sum of salaries for male and female.
  
  Args:
    salaries: List of tuple(is male, salary)
    
  Returns:
    tuple(sum of male's salary, sum of female's salary)
  """
  male_sum = sum([t[1] for t in salaries if t[0]])
  female_sum = sum([t[1] for t in salaries if not t[0]])
  return (male_sum, female_sum)

In [4]:
declarative_way(salaries)

(23000, 18000)

In [5]:
from fpu.flist import fl

def declarative_way_v2(salaries):
  """Calculates the sum of salaries for male and female.
  
  Args:
    salaries: List of tuple(is male, salary)
    
  Returns:
    tuple(sum of male's salary, sum of female's salary)
  """
  def add(a, b): return a + b
  def is_male(t): return t[0]
  def is_female(t): return not t[0]
  im_list = fl(salaries)
  male_sum = im_list.filter(is_male) \
                    .map(lambda t: t[1]) \
                    .foldLeft(0, add)
  female_sum = im_list.filter(is_female) \
                      .map(lambda t: t[1]) \
                      .foldLeft(0, add)
  return (male_sum, female_sum)

In [6]:
declarative_way_v2(salaries)

(23000, 18000)

In [7]:
def salary_sum(is_male: bool = True):
  def sexual_check(t): return t[0] == is_male
  def _salary_sum(salaries):
    def add(a, b): return a + b
    im_list = fl(salaries)
    return im_list.filter(sexual_check) \
                  .map(lambda t: t[1]) \
                  .foldLeft(0, add)
  
  return _salary_sum
  
  
def declarative_way_v3(salaries):
  """Calculates the sum of salaries for male and female.
  
  Args:
    salaries: List of tuple(is male, salary)
    
  Returns:
    tuple(sum of male's salary, sum of female's salary)
  """
  return (
    salary_sum(True)(salaries),
    salary_sum(False)(salaries),
  )

In [8]:
declarative_way_v3(salaries)

(23000, 18000)

## <font color='darkblue'>FP Terminology (Closures/Curring)</font>
A [**Closure**](https://en.wikipedia.org/wiki/Closure_(computer_programming)) is a function which **simply creates a scope that allows the function to access and manipulate the variables in enclosing scopes**. Normally, you will follow below steps to create a Closure in Python:
* We have to create a nested function (a function inside another function).
* This nested function has to refer to a variable defined inside the enclosing function.
* The enclosing function has to return the nested function

In [9]:
data = [1, 2, 3, 4, 5]

def contain_n(n):
  def closures(alist):
    return n in alist
  
  return closures

In [10]:
is_5_exist = contain_n(5)
is_10_exist = contain_n(10)

print(f'{data}: is_5_exist? {is_5_exist(data)}')
print(f'{data}: is_10_exist? {is_10_exist(data)}')

[1, 2, 3, 4, 5]: is_5_exist? True
[1, 2, 3, 4, 5]: is_10_exist? False


### <font color='darkgreen'>Why Closures</font>
Even if closures seem pretty interesting (a function returning another function which knows its creation context!) there is another question: where **can we utilize closures to make the best of them? Here are a few uses for closures**:
* Eliminating global variables
* Replacing hard-coded constants
* Providing consistent function signatures

> No, a function inside of a function doesn’t have to reference variables outside of its scope. A closure only exists when a function accesses a variable(s) outside of its immediate scope.


### <font color='darkgreen'>Currying</font>
<b><font size='3ptx' color='darkblue'>[Currying](https://en.wikipedia.org/wiki/Currying)</font> is like a kind of incremental binding of function arguments</b>. It is the technique of breaking down the evaluation of a function that takes multiple arguments into evaluating a sequence of single-argument functions:
* Concept by Haskell Curry
* Translating a function that takes multiple arguments into a sequence of functions which all take 1 argument. e.g.: `add(a, b)` AND `add(a)(b)`
* Improves reusability and composition
* In some languages (Haskell, F#) functions are curried by default


In [13]:
def curry_ex(a):
  def inner(b):
    return a + b
  
  return inner

  
print(f'1 + 2 = {curry_ex(1)(2)}')
print(f'2 + 3 = {curry_ex(2)(3)}')

1 + 2 = 3
2 + 3 = 5


However, Python doesn't support [Currying](https://en.wikipedia.org/wiki/Currying) well. So you have to use decorator technique to get around it. One example:

In [16]:
from inspect import signature

def curry(x, argc=None):
  if argc is None:
    argc = len(signature(x).parameters)
    
  def p(*a):
    if len(a) == argc:
      return x(*a)
    
    def q(*b):
      return x(*(a + b))
    
    return curry(q, argc - len(a))
  
  return p

In [17]:
@curry
def my_func(a, b, c):
  print(f'{a}-{b}-{c}')

In [19]:
a = 1; b = 2; c = 3
my_func(a)(b)(c)
my_func(a, b, c)
my_func(a, b)(c)

1-2-3
1-2-3
1-2-3


## <font color='darkblue'>FP Terminology (Partial application)</font>
<b><font size='3ptx' color='blue'>[Partial application](https://en.wikipedia.org/wiki/Partial_application)</font> allow one to derive a function with x parameters to a function with fewer parameters and fixed values set for the more limited function</b>. Module [**functools**](https://docs.python.org/3/library/functools.html) offers some tools for the functional approach.

In [22]:
import functools

def say_greeting_to(name: str, greeting: str):
  print(f'{greeting}, {name}!')
  
say_hello_to = functools.partial(say_greeting_to, greeting='hello')
greet_peter = functools.partial(say_greeting_to, name='Peter')

In [25]:
say_hello_to('John')
say_hello_to('Ken')

hello, John!
hello, Ken!


In [26]:
greet_peter(greeting='Hi')
greet_peter(greeting='Aloha')

Hi, Peter!
Aloha, Peter!


## <font color='darkblue'>FP Terminology (Recursion)</font>
<b>FP favors <font size='3ptx' color='blue'>Recursion</font> over for/while loop</b>. The best example to adopt recursion is to calculate [**factorial**](https://en.wikipedia.org/wiki/Factorial) (e.g.: `5!=5*4*3*2*1`): 

In [27]:
import sys

print(f'Recursion limit={sys.getrecursionlimit()}')

Recursion limit=3000


In [51]:
def triangular_number_loop(n: int):
  v = 1
  for i in range(2, n+1):
    v += i
    
  return v

# (1 + 5) * 5 / 2 = 15
print(f'triangular_number_loop(5)={triangular_number_loop(5)}')

triangular_number_loop(5)=15


In [52]:
def triangular_number_recv(n: int):
  if n == 1: return 1
  return n + triangular_number_recv(n - 1)

print(f'triangular_number_recv(5)={triangular_number_recv(5)}')

triangular_number_recv(5)=15


In [54]:
# RecursionError: maximum recursion depth exceeded in comparison
# triangular_number_recv(3001)

## <font color='darkblue'>FP Terminology (Lazy Evaluation)</font>
When applied to method arguments, <b><font size='3ptx' color='blue'>strictness</font></b> means that arguments are evaluated as soon as they’re received by the method. <b><font size='3ptx' color='blue'>Laziness</font> means that arguments are evaluated only when they’re needed</b> (<font color='brown'>Lazy Evaluation</font>). Of course, strictness and laziness apply not only to method arguments, but to everything:
* Iterators and generators
* Saves memory and possibly CPU time
* Working with infinite data structures

![joke](images/1.PNG)


Let’s take a look at first usage of “Lazy Evaluation”:


In [66]:
from fpu.fp import Supplier

class John:
  def __init__(self):
    self.tired = True
    self.bored = False

john = John()
    
def am_tired():
  return john.tired


def am_bored():
  raise Exception('Blow up')


def do_or(cond1, cond2):
  v1 = cond1()
  v2 = cond2()  # Blow up here
  return v1 or v2
  

def lazy_or(cond1, cond2):
  return cond1() or cond2()  # Because of short cut, cond2() won't be executed


def my_next_move(bored_or_tired):
  if bored_or_tired:
    return "Go home"
  else:
    return "Hanging around"

In [64]:
# Exception: Blow up
# my_next_move(do_or(am_tired, am_bored))

In [65]:
my_next_move(lazy_or(am_tired, am_bored))

'Go home'

Let's take a look at [triangular number](https://en.wikipedia.org/wiki/1_%2B_2_%2B_3_%2B_4_%2B_%E2%8B%AF) example again:

In [56]:
from fpu.fp import *

ret = TailCall.ret; sus = TailCall.sus

def triangular_number_lazy(n, x=1):
  return ret(x) if n == 1 else sus(Supplier(factorial_lazy, n-1, x + n))

In [60]:
triangular_number_lazy(5)  # Lazy evaluation. It will execute until we call `eval()`

In [57]:
triangular_number_lazy(5).eval()

15

In [58]:
factorial_lazy(3001).eval()

4504501

Another usage of “Lazy Evaluation” is for infinite data structures ( [**Generator**](https://realpython.com/introduction-to-python-generators/) & [**Iterator**](https://realpython.com/python-iterators-iterables/)):

In [67]:
rng_1_to_1000 = range(1, 1001)

In [73]:
it = rng_1_to_1000.__iter__()
print(it.__next__())
print(it.__next__())

1
2


## <font color='darkblue'>Built-in FP in Python (filter, reduce and map)</font>
Here we will going to introduce some built-in FP functions in Python.
![filter, map](images/2.PNG)
![reduce](images/3.PNG)

In [81]:
from functools import reduce

food_list = [
  {'name': 'beef', 'is_veg': False},
  {'name': 'potato', 'is_veg': True},
  {'name': 'chicken', 'is_veg': False},
  {'name': 'corn', 'is_veg': True},
]

def get_name(food):
  return food['name']


def is_veg(food):
  return food['is_veg']


def cook(food):
  if food['name'] == 'beef':
    return 'Hambuger'
  if food['name'] == 'potato':
    return 'French fries'
  if food['name'] == 'chicken':
    return 'Fried chicken'
  if food['name'] == 'corn':
    return 'Pop corn'
  
  raise Exception('Unknown')

In [78]:
list(map(cook, food_list))

['Hambuger', 'French fries', 'Fried chicken', 'Pop corn']

In [82]:
list(map(get_name, filter(is_veg, food_list)))

['potato', 'corn']

In [86]:
def add(a, b): return a + b
reduce(add, range(1, 6), 0)

# add(0, 1) = 1
# add(1, 2) = 3
# add(3, 3) = 6
# add(6, 4) = 10
# add(10, 5) = 15

15

## <font color='darkblue'>Real world examples</font>

### <font color='darkgreen'> HackerRank Sample (`gem-stones`) </font>
([Problem source](https://www.hackerrank.com/challenges/gem-stones/problem))
![gem-stones problem](images/4.PNG)

In [95]:
# gem `a` and `b` exist in all three rocks
data = ('abcdde', 'baccd', 'eeabg')

#### <b>Imperative approach</b>

In [93]:
def gem_stones_imp(data):
  set_list = []
  # 1) Collect unique element of each rock
  for s in data:
    set_list.append(set(list(s)))
    
  # 2) Keep finding common gems in each rock
  common_set = None
  for a_set in set_list:
    if common_set is None:
      common_set = a_set
      continue
      
    common_set &= a_set
    
  return len(common_set)

In [94]:
print(f'Output of gem_stones_imp={gem_stones_imp(data)}')

Output of gem_stones_imp=2


#### <b>Declarative approach</b>

In [113]:
from fpu.flist import *

def gem_stones_dec(data):
  rlist = fl(list(data))
  return len(
    rlist.map(set)                     # Turn list into set. e.g. ['a', 'b', 'b'] -> {'a', 'b'}
         .reduce(lambda a, b: a & b)   # Iteratively find intersection of each set
  )                                    # Return the length of final set

In [114]:
gem_stones_dec(data)

2